In [1]:
"""

Notes on this model: https://shizacharania.notion.site/FCN-Fully-Convolutional-Network-bbc70d47ad92491cb534e2987ac7eb48

class convBlock (because you have 6 so its more feasible)
  - conv3d 3x3 (increase channels)
  - relu
  - conv3d 3x3 (same channels)
  - relu
  - max pool = conv with same channels, but stride is 2 and kernel is 2 (just like max pool layer)

  ~~~~~~~~~~~~~~~~~~

  - convtranspose3d = deconv increased channels, but stride is 2 and kernel is 3 (combining steps) - this is approach one
 
  ~~~~~~~~~~~~~~~~~~

  approach 2:
  - convtranspose3d = deconv with same channels, but stride is 2 and kernel is 2 (to increase spatial size)
  - conv 3x3 (less channels)

  ~~~~~~~~~~~~~~~~~~

  torch.add - for element wise summation
"""

'\n\nNotes on this model: https://shizacharania.notion.site/FCN-Fully-Convolutional-Network-bbc70d47ad92491cb534e2987ac7eb48\n\nclass convBlock (because you have 6 so its more feasible)\n  - conv3d 3x3 (increase channels)\n  - relu\n  - conv3d 3x3 (same channels)\n  - relu\n  - max pool = conv with same channels, but stride is 2 and kernel is 2 (just like max pool layer)\n\n  ~~~~~~~~~~~~~~~~~~\n\n  - convtranspose3d = deconv increased channels, but stride is 2 and kernel is 3 (combining steps) - this is approach one\n \n  ~~~~~~~~~~~~~~~~~~\n\n  approach 2:\n  - convtranspose3d = deconv with same channels, but stride is 2 and kernel is 2 (to increase spatial size)\n  - conv 3x3 (less channels)\n\n  ~~~~~~~~~~~~~~~~~~\n\n  torch.add - for element wise summation\n'

In [2]:
import torch
import torch.nn as nn

In [3]:
class ConvBlockEncoder(nn.Module): # need to add nn.Module here
  def __init__(self, input_channels, output_channels): # need to add variables needed here
    super().__init__()
    self.convblock1 = nn.Conv3d(in_channels=input_channels, out_channels=output_channels, kernel_size=3, padding=1)
    self.relu = nn.ReLU()
    self.convblock2 = nn.Conv3d(in_channels=output_channels, out_channels=output_channels, kernel_size=3, padding=1)
    self.downsample = nn.Conv3d(in_channels=output_channels, out_channels=output_channels, kernel_size=2, stride=2)

  def forward(self, x): # x represents the input
    x = self.convblock1(x)
    x = self.relu(x)
    x = self.convblock2(x)
    out = self.relu(x)
    down_out = self.downsample(out)
    return out, down_out # out is before downsamping and down_out is after

class ConvBlockDecoder(nn.Module):
  def __init__(self, input_channels, output_channels):
    super().__init__()
    # self.deconv = nn.ConvTranspose3d(in_channels=input_channels, out_channels=output_channels, kernel_size=3, stride=2, padding=1, dilation=1)
    self.upsample = nn.ConvTranspose3d(in_channels=input_channels, out_channels=input_channels, kernel_size=2, stride=2)
    self.convblock = nn.Conv3d(in_channels=input_channels, out_channels=output_channels, kernel_size=3, padding=1)

  def forward(self, x):
    # x = self.deconv(x) # didn't work because it was just outputing 7 or 9 as the amount of channels and I coulnd't change the padding because it would either have one extra or one less (and I wanted 8 as the spatial dimensions)
    x = self.upsample(x)
    x = self.convblock(x)
    return x


class FCN(nn.Module):
  def __init__(self, input_shape):
    super().__init__()
    # 4, 128, 128, 128
    print(input_shape)
    self.nimages, self.channels, self.w, self.h, self.d = input_shape
    self.block1 = ConvBlockEncoder(4, 64)
    self.block2 = ConvBlockEncoder(64, 128)
    self.block3 = ConvBlockEncoder(128, 256)
    self.block4 = ConvBlockEncoder(256, 512)
    self.block5 = ConvBlockEncoder(512, 1024)
    self.block6 = ConvBlockEncoder(1024, 2048)

    self.block7 = ConvBlockDecoder(2048, 1024)
    self.block8 = ConvBlockDecoder(1024, 512)
    self.block9 = ConvBlockDecoder(512, 256)
    self.block10 = ConvBlockDecoder(256, 128)
    self.block11 = ConvBlockDecoder(128, 64)

    self.conv1by1 = nn.Conv3d(in_channels=64, out_channels=3, kernel_size=1, stride=1) # output is 3 channels for WT, ET, TC
    self.softmax = nn.Softmax(dim=1)
  
  def forward(self, x):
    print("Before shape: ", str(x.shape))
    out1, down_out1 = self.block1(x)
    print(out1.shape, down_out1.shape)

    out2, down_out2 = self.block2(down_out1)
    print(out2.shape, down_out2.shape)

    out3, down_out3 = self.block3(down_out2)
    print(out3.shape, down_out3.shape)

    out4, down_out4 = self.block4(down_out3)
    print(out4.shape, down_out4.shape)

    out5, down_out5 = self.block5(down_out4)
    print(out5.shape, down_out5.shape)

    out6, ____ = self.block6(down_out5)
    print(out6.shape)

    out7 = self.block7(out6)
    print(out7.shape)

    # sum here with block 5 + block 7
    out57 = torch.add(out5, out7)
    print(out57.shape)

    out8 = self.block8(out57)
    print(out8.shape)
    
    # sum here with block 4 + block 8
    out48 = torch.add(out4, out8)
    print(out48.shape)

    out9 = self.block9(out48)
    print(out9.shape)

    out10 = self.block10(out9)
    print(out10.shape)

    out11 = self.block11(out10)
    print(out11.shape)

    out12 = self.conv1by1(out11)
    print(out12.shape)

    end = self.softmax(out12)
    print(end.shape) # i thought that the shape would change when using softmax, but only the values change to add up to 1

    return out12, end

In [4]:
x = torch.rand(size=(1, 4, 128, 128, 128), dtype=torch.float32)
# print(x.shape)

model = FCN(x.shape)
# print(model)
# print()

# out = model(x)
# print(out.shape)

torch.Size([1, 4, 128, 128, 128])


In [5]:
# output, probability = out

In [6]:
# print(output.shape)
# print(output)

In [7]:
# print(probability.shape)
# print(probability)

In [8]:
"""
- SGD
- minibatch size = 20
- learning rates of 10^-3 (fcn-alexNet), **10^-4 (fcn-vgg16- use this initially because the paper said the model is “our VGG-16 based net)**, ^-5 (fcn-googlelenet)
- momentum = 0.9
- weight decay = 5^-4 or 2^-4

- dropout was included where used in the original classifier nets
- didn’t choose patches because although it has a better convergence rate compared to the whole image training, it takes significantly more time because of the larger numbers that need to be considered per batch
- data augmentation of randomly mirroring and “jittering” the images by translating them up to 32 pixels
- Nvidia Tesla K40 GPU
- metrics (testing)
"""

'\n- SGD\n- minibatch size = 20\n- learning rates of 10^-3 (fcn-alexNet), **10^-4 (fcn-vgg16- use this initially because the paper said the model is “our VGG-16 based net)**, ^-5 (fcn-googlelenet)\n- momentum = 0.9\n- weight decay = 5^-4 or 2^-4\n\n- dropout was included where used in the original classifier nets\n- didn’t choose patches because although it has a better convergence rate compared to the whole image training, it takes significantly more time because of the larger numbers that need to be considered per batch\n- data augmentation of randomly mirroring and “jittering” the images by translating them up to 32 pixels\n- Nvidia Tesla K40 GPU\n- metrics (testing)\n'

In [9]:
randomized_training_images = []
for i in range(2):
  newx = torch.rand(size=(1, 4, 128, 128, 128), dtype=torch.float32)
  randomized_training_images.append(newx)

In [10]:
print(len(randomized_training_images))
print(randomized_training_images[0].shape)
# print(randomized_training_images[0])

2
torch.Size([1, 4, 128, 128, 128])


In [11]:
import random
randomized_training_segmentations = []
for i in range(2):
  newy = torch.rand(size=(1, 128, 128, 128), dtype=torch.float32)
  randomized_training_segmentations.append(newy)

In [12]:
print(len(randomized_training_segmentations))
print(randomized_training_segmentations[0].shape)
# print(randomized_training_segmentations[1])

2
torch.Size([1, 128, 128, 128])


In [13]:
randomized_training_data = list(zip(randomized_training_images, randomized_training_segmentations))

In [14]:
trainloader = torch.utils.data.DataLoader(dataset=randomized_training_data, batch_size=2, shuffle=True) # minibatch size should be 20

In [15]:
len(trainloader)

1

In [16]:
import torch.optim

In [17]:
# epochs
epochs = 1
# loss
criterion = torch.nn.CrossEntropyLoss()
# optimizer
optimizer = torch.optim.SGD(params=model.parameters(), lr=10**-4, momentum=0.9, weight_decay=5**-4)

In [18]:
training_losses = []

for i in range(epochs):
  training_loss = 0
  for images, segs in trainloader:
    optimizer.zero_grad()
    print(len(images), len(segs))
    print(images.shape)
    print(segs.shape)
    images = images.squeeze()
    images = torch.tensor(data=images, dtype=torch.float, requires_grad=True)
    # segs = segs.long() - no
    segs = segs.squeeze()
    segs = torch.tensor(data=segs, requires_grad=True)
    print(images.shape)
    print(segs.shape)
    print()
    outputs = model(images)
    outs, softmax_outs = outputs
    print()
    print(softmax_outs.shape)
    softmax_outs = torch.argmax(softmax_outs, dim=1)
    print(softmax_outs.shape)
    print()
    loss = criterion(softmax_outs.float(), segs)
    print(loss)
    loss.backward()
    training_loss += loss.item()
    # break
  training_losses.append(training_loss/len(trainloader))

print(training_losses)

2 2
torch.Size([2, 1, 4, 128, 128, 128])
torch.Size([2, 1, 128, 128, 128])
torch.Size([2, 4, 128, 128, 128])
torch.Size([2, 128, 128, 128])

Before shape:  torch.Size([2, 4, 128, 128, 128])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


torch.Size([2, 64, 128, 128, 128]) torch.Size([2, 64, 64, 64, 64])
torch.Size([2, 128, 64, 64, 64]) torch.Size([2, 128, 32, 32, 32])
torch.Size([2, 256, 32, 32, 32]) torch.Size([2, 256, 16, 16, 16])
torch.Size([2, 512, 16, 16, 16]) torch.Size([2, 512, 8, 8, 8])
torch.Size([2, 1024, 8, 8, 8]) torch.Size([2, 1024, 4, 4, 4])
torch.Size([2, 2048, 4, 4, 4])
torch.Size([2, 1024, 8, 8, 8])
torch.Size([2, 1024, 8, 8, 8])
torch.Size([2, 512, 16, 16, 16])
torch.Size([2, 512, 16, 16, 16])
torch.Size([2, 256, 32, 32, 32])
torch.Size([2, 128, 64, 64, 64])
torch.Size([2, 64, 128, 128, 128])
torch.Size([2, 3, 128, 128, 128])
torch.Size([2, 3, 128, 128, 128])

torch.Size([2, 3, 128, 128, 128])
torch.Size([2, 128, 128, 128])

tensor(310.5766, grad_fn=<DivBackward1>)
[310.5766296386719]
